<a href="https://colab.research.google.com/github/m7mdsayied/gemma3-finetune/blob/main/Working_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes mistral-common

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/gemma-3-4b-it"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import Dataset
import json


tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

# Load your data
with open("generated_level4_jobs.json", "r") as f:
    raw_data = json.load(f)

# Define instruction
INSTRUCTION = "Analyze the following job title and description and output its ISCO-08 Code (4-digit level), the original ISCO-08 Job Title, and a justification for this classification in a JSON format with no markdown. Output ONLY valid JSON."

def create_conversations_dataset(raw_data):
    processed_examples = []
    for entry in raw_data:
        isco_code = entry["isco_code"]
        original_title = entry["original_title"]
        for job in entry["generated_jobs"]:
            new_job_title = job["new_job_title"]
            generated_job_description = job["generated_job_description"]
            justification = job["justification"]

            user_input = f"{INSTRUCTION}\nJob Title: {new_job_title}\nJob Description: {generated_job_description}"
            model_output_json = json.dumps({
                "isco_code": isco_code,
                "original_title": original_title,
                "justification": justification
            }, indent=None)

            conversation = [
                {"role": "user", "content": user_input},
                {"role": "model", "content": model_output_json}
            ]
            processed_examples.append({"conversations": conversation})

    return processed_examples

processed_data = create_conversations_dataset(raw_data)
dataset = Dataset.from_list(processed_data)
print(f"Created dataset with {len(dataset)} examples")

Created dataset with 2160 examples


In [ ]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/2160 [00:00<?, ? examples/s]

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import train_on_responses_only

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

# Train only on responses
trainer = train_on_responses_only(
    trainer,
    instruction_part="<start_of_turn>user\n",
    response_part="<start_of_turn>model\n",
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2160 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2160 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,160 | Num Epochs = 1 | Total steps = 270
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 131,153,920 of 4,431,233,392 (2.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,1.986500
50,1.533700
75,1.475000
100,1.442000
125,1.432500
150,1.373100
175,1.385400
200,1.323000
225,1.343600
250,1.309400


In [ ]:
# To test inference

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
instruction = "Analyze the following job title and description and output its ISCO-08 Code (4-digit level), the original ISCO-08 Job Title, and a justification for this classification in a JSON format with no markdown. Output ONLY valid JSON."
test_job_title = "Municipal Budget Reviewer"
test_job_description = "Reviews and scrutinizes local government department budgets to ensure fiscal responsibility and alignment with city council mandates. Prepares presentation materials for senior policy makers on budget effectiveness."

user_content = f"{instruction}\nJob Title: {test_job_title}\nJob Description: {test_job_description}"

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : user_content,}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 0.8, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nAnalyze the following job title and description and output its ISCO-08 Code (4-digit level), the original ISCO-08 Job Title, and a justification for this classification in a JSON format with no markdown. Output ONLY valid JSON.\nJob Title: Municipal Budget Reviewer\nJob Description: Reviews and scrutinizes local government department budgets to ensure fiscal responsibility and alignment with city council mandates. Prepares presentation materials for senior policy makers on budget effectiveness.<end_of_turn>\n<start_of_turn>model\n{"isco_code": "2241", "original_title": "Government Officials", "justification": "This role focuses on the \'coordinating and reviewing government plans, programmes and budgets\' (a) and \'preparing and directing the preparation of reports, presentations and other material\' (b) aspects, specifically within the context of budget management and presentation. It is a specialized, less direct, and more analytical function."}<end_of_tur

In [ ]:
# Merge base model with LoRA
model = model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# Built-in model.save_pretrained_gguf() & unsloth_convert_hf_to_gguf.py still doesn't work on Gemma-3 and new models
# Use the official convert-hf-to-gguf.py script from llama.cpp, which supports more architectures and is updated regularly.

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch, shutil
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

BASE_MODEL = "unsloth/gemma-3-4b-it"
LORA_PATH = "/content/outputs/checkpoint-270"  # your LoRA folder
SAVE_HF = "/content/merged_hf"
SAVE_GGUF = "/content/merged_gguf"
DRIVE_GGUF = "/content/drive/MyDrive/merged_gguf"

# Load base and merge LoRA
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16, device_map="auto")
model = PeftModel.from_pretrained(base, LORA_PATH)
merged = model.merge_and_unload()

# Save HF format
merged.save_pretrained(SAVE_HF)
AutoTokenizer.from_pretrained(BASE_MODEL).save_pretrained(SAVE_HF)
print(f"✅ Merged model saved in HF format at {SAVE_HF}")


Mounted at /content/drive


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ Merged model saved in HF format at /content/merged_hf


In [ ]:
# Get llama.cpp if not cloned
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp


fatal: destination path 'llama.cpp' already exists and is not an empty directory.
/content/llama.cpp


In [ ]:
!python3 /content/llama.cpp/convert_hf_to_gguf.py \
  /content/merged_hf \
  --outfile /content/merged_gguf/gemma3_finetuned.gguf \
  --outtype q8_0


INFO:hf-to-gguf:Loading model: merged_hf
INFO:hf-to-gguf:Model architecture: Gemma3ForConditionalGeneration
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,                 torch.float16 --> Q8_0, shape = {2560, 262208}
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.float16 --> F32, shape = {2560}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.float16 --> Q8_0, shape = {10240, 2560}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,             torch.float16 --> Q8_0, shape = {2560, 10240}
INFO:hf-to-gguf:blk.0.ffn_up.weight,               torch.float16 --> Q8_0, shape = {2560, 10240}
INFO:hf-to-gguf:blk.0.post_attention_norm.weight,  torch.float16 --> F32, shape = {2560}
INFO:hf-to-gguf:blk.0.post_ffw_norm.weight,        torch.float

In [ ]:
import shutil, os
os.makedirs("/content/drive/MyDrive/merged_gguf", exist_ok=True)
shutil.copy(
    "/content/merged_gguf/gemma3_finetuned.gguf",
    "/content/drive/MyDrive/merged_gguf/gemma3_finetuned.gguf"
)
print("📦 GGUF model copied to Drive successfully!")


📦 GGUF model copied to Drive successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
